In [3]:
# http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [11]:
# http://www.voynich.nu/transcr.html#FSG
! wget -nc http://www.ic.unicamp.br/~stolfi/voynich/mirror/reeds/docs/FSG.txt

Plik `FSG.txt' już istnieje, bez pobierania.



In [5]:
import numpy
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using Theano backend.


In [6]:
# prepare input
vm_filename = "FSG_cleaned_small.txt" # "FSG_cleaned_small.txt"
vm_text = open(vm_filename).read()

# convert characters the to integers
chars = sorted(list(set(vm_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(vm_text)
n_vocab = len(char_to_int)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  6222
Total Vocab:  27


In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []
for i in range(0, len(vm_text) - seq_length, 1):
	seq_in = vm_text[i:i + seq_length]
	seq_out = vm_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  6182


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
# define the LSTM model
# simple model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# define the checkpoint
filepath="simple_model_40_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X, y, callbacks=callbacks_list, batch_size=128, nb_epoch=20)

In [ ]:
# Generator

In [22]:
import sys
# load the network weights
filename = "simple_model_40_weights-improvement-19-1.0051.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
int_to_char = {v: k for k, v in char_to_int.items()}

In [24]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(300):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result + "")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" DTG,DTOK,TOR,TCDCOE,ODA-
GHTCCAR,ODCOE,H "
ZOE,8AM,HZOE-
GTOR,TOE,HZOE,TOH,TO8G,TO8G,TOE,ODTOE,8AM-
SO,TOE,SOE,HZOE,SO8A,TOE,8AM,OHTOE,8AM-
SO,TOE,SOE,HZOE,SO8AN,TOE,8AM,HZOE,8AM-
SOR,TOE,HZOE,8AM,HZOE,8AM,HZOE,8AM,HZOR,TOHZG-
4TOR,TOE,HZOE,TOH,TOE,HZOE,8AM,HZOE,8AM,
ODTO,GZG,DTOR,TOE,O8G,8OE,8A,8OE,TO8G-
ODOE,TOE,SOE,8AM,ODTOE,8AIIR,TOR,OHA
Done.
